# Single Nuclear RNAseq Processing

In [ ]:
import snRNAseq_processing_FUNCTIONS as myfunc
import pandas as pd
import scanpy as sc
import anndata as ad
import numpy as np
import scanpy.external as sce
import scrublet as scr
import scipy.sparse as sparse


import random
# Setting seed
random.seed(11)

## 1. Data Processing

###  A. Load Data and Metadata

In [ ]:
meta_data = pd.read_csv('mtg_meta_data.csv')
adatas=myfunc.read_h5_files(meta_data='mtg_meta_data.csv',base_dir='/tscc/lustre/ddn/scratch/aopatel/mtg_h5',filtered=True)

<div class="alert alert-block alert-info">
<b> CHECK DUPLICATED GENES. In the MTG the duplicated gene symbols are MKKS', 'DNAJC9-AS1', 'DDX11L16', 'TNFRSF10A-DT', 'LINC01605', 'LINC02256', 'LSP1P5', 'RAET1E-AS1', 'LINC03025', 'NPIPA9', 'PRICKLE2-AS1', 'ARMCX5-GPRASP2', 'SPATA13', 'ELFN2', 'LINC01238', 'GPR84-AS1', 'LINC00484', 'LINC03023', 'LINC03021', 'LINC01115', 'GOLGA8M. They are duplicated in 127/127 samples. We will not change them.

</div>

In [ ]:
for i, adata in enumerate(adatas):
    if adata.var_names.is_unique:
        print(f"AnnData object {i}: Variable names are unique.")
    else:
        counts = adata.var_names.value_counts()
        duplicates = counts[counts > 1]
        print(f"AnnData object {i}: Variable names are not unique. Duplicated names: {list(duplicates.index)}")

<div class="alert alert-block alert-info">
<b> Check sizes of each object in adatas, ensure number of "genes" is same and in the correct gene id format. In the MTG, all samples have 38,606 gene symbols

</div>


In [ ]:
for index, item in enumerate(adatas):
    print(f"Index: {index}, Value: {item}")

### B. Add Metadata to each sample (object in adatas[] list)

In [ ]:
for i, adata in enumerate(adatas):
    # Store the barcodes (index) as a column before merging
    adata.obs['barcode'] = adata.obs.index
    
    # Add the index column with the iteration number
    adata.obs['index'] = i
    
    # Merge the metadata with adata based on '10X_ID'
    adata.obs = adata.obs.merge(meta_data, on='10X_ID', how='left')
    
    # After merging, restore the barcodes as the index
    adata.obs.set_index('barcode', inplace=True)
    
    print(f"Metadata added for sample {i + 1}")

In [ ]:
# Check
print(adatas[1].obs)

### C. Change index column of each object in adatas[] to gene_id

In [ ]:
# Modify each AnnData object to use gene_ids as index and store gene symbols
for adata in adatas:
    # Ensure gene_ids and gene symbols exist in .var
    if 'gene_ids' not in adata.var.columns or adata.var.index.name != 'gene_symbols':
        # Store current index (gene symbols) as a column
        adata.var['gene_symbols'] = adata.var.index
        # Set gene_ids as the new index
        adata.var.set_index('gene_ids', inplace=True)

In [ ]:
adatas[120].var

## 2. Quality Control (QC) and Final Merge

### A. Initial view and assessment

In [ ]:
adatas=myfunc.pre_QC_view(adatas)

### B. Filtering

In [ ]:
filtered_adatas, summary_df = myfunc.QC_filtering(adatas=adatas,
                                                   min_genes=200,  #minimum number of non-zero valued genes a cell must have to be kept
                                                   mt_thresh=5, ribo_thresh=5, hb_thresh=1) 

In [ ]:
pd.set_option('display.max_rows', None) 
# Show all rows pd.set_option('display.max_columns', None) # Show all columns pd.set_option('display.max_colwidth', None) # Show full column content pd.set_option('display.width', None) # Use full terminal width
summary_df.to_csv('summary_df_mtg.csv', index=False) 
summary_df

In [ ]:
# Check normalization status again
myfunc.check_normalization_status(adatas)

#### B1. Check for Ambient RNA (Not Required!)

In [ ]:
#myfunc.view_ambient(filtered_adatas)

In [ ]:
#filtered_adatas=myfunc.ambient_removal(filtered_adatas)

### C. Doublet Removal

In [ ]:
filtered_adatas, doublet_summary_df = myfunc.run_scrublet(filtered_adatas,expected_doublet_rate=0.06)


### D. Concatenation

In [ ]:
# Concatenate all datasets into 1
merged_adata = sc.concat(filtered_adatas, join='outer', index_unique="-") #index is for barcodes that may similar beteen samples

In [ ]:
merged_adata

In [ ]:
### re-add gene annotation (lost during concat) back 
var_dfs = [adata.var for adata in filtered_adatas]
merged_var = pd.concat(var_dfs, axis=0, join='outer')

# Combine values for  gene_ids
merged_var = merged_var.groupby(level=0).first()

# Assign merged .var to merged AnnData
merged_adata.var = merged_var

In [ ]:
sc.pp.filter_genes(merged_adata, min_cells=5) #filter genes that are not in at least X cells
print(f"Number of genes after filtering: {merged_adata.shape[1]}")

In [ ]:
print(merged_adata)

<div class="alert alert-block alert-info">
<b> Save the final file (that we are going to use for clustering and analysis!)

</div>


In [ ]:
# Save progress up to this point, just in case
merged_adata.write_h5ad("/tscc/lustre/ddn/scratch/aopatel/fin_adata_mtg.h5ad")

### F. Check UMAP prior to any Batch Correction

In [ ]:
##### Create layer that is not manipulated
adata.layers["counts"] = adata.X.copy()

##### Normalize, log transform and scale

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.scale(adata, max_value=10)

##### Find highly variable genes using "seurat_v3", takes raw data only, provide non-manipulated layer
sc.pp.highly_variable_genes(adata, n_top_genes=3000, flavor="seurat_v3", layer="counts",  batch_key='libraryBatch')

##### PCA
sc.tl.pca(adata, n_comps=50, use_highly_variable=True)
sc.pl.pca_variance_ratio(adata, log=True, n_pcs= 50)

In [ ]:
##### Find neighbors, use leiden clustering, and crate umap
sc.pp.neighbors(adata,n_neighbors=15,random_state=11)  #n_neighbors=30 can be used as well
sc.tl.leiden(adata, resolution=0.50, key_added='leiden',random_state=11)  # Adjust resolution as needed

# Changed min_dist for optimal graphing
sc.tl.umap(adata, random_state=11, min_dist=0.15)

## 3. Batch correction and integration (Harmony)

### A. Harmony

#### A1. Set up and perform Harmony integration/batch correction

In [ ]:
sc.settings.verbosity = 1  # Set to show only warnings and errors

In [ ]:
adata = ad.read_h5ad("/tscc/lustre/ddn/scratch/aopatel/fin_adata_mtg.h5ad")
adata

<div class="alert alert-block alert-info">
<b> Filter for protein coding genes only

</div>


In [ ]:
adata = protein_coding_genes(adata)

<div class="alert alert-block alert-info">
<b> Make sure some genes of interest are still included

</div>


In [ ]:
genes_of_interest = ['REST', 'OLIG2', 'VIP', 'SUZ12', 'NRF1','SST','MAP2','SNAP25','VIP', 'SST', 'CHODL']

# Boolean mask for genes present in adata.var['gene_symbols']
mask = adata.var['gene_symbols'].isin(genes_of_interest)

# Genes that are present
present_genes = adata.var.loc[mask, 'gene_symbols'].tolist()

# Genes that are missing
missing_genes = [g for g in genes_of_interest if g not in present_genes]

print("Present genes:", present_genes)
print("Missing genes:", missing_genes)

<div class="alert alert-block alert-info">
<b> Start preparing for UMAP

</div>

In [ ]:
##### Create layer that is not manipulated
adata.layers["counts"] = adata.X.copy()

##### Normalize, log transform and scale

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.scale(adata, max_value=10)

##### Find highly variable genes using "seurat_v3", takes raw data only, provide non-manipulated layer
sc.pp.highly_variable_genes(adata, n_top_genes=3000, flavor="seurat_v3", layer="counts",  batch_key='libraryBatch')

##### PCA
sc.tl.pca(adata, n_comps=50, use_highly_variable=True)
sc.pl.pca_variance_ratio(adata, log=True, n_pcs= 50)

In [ ]:
sc.pl.pca(
    adata,
    color=["Consensus clinical diagnosis"],
    dimensions=[(0, 1)],
    size=2,
)

<div class="alert alert-block alert-info">
<b> Slice the PCA embedding to use only the first 20 PCs for Harmony

</div>

In [ ]:
adata.obsm['X_pca_harmony_input'] = adata.obsm['X_pca'][:, :20]

<div class="alert alert-block alert-info">
<b> Begin Harmony

</div>

In [ ]:
############################# HARMONY #######################################
sce.pp.harmony_integrate(adata,
                         key=['individualID','libraryBatch','Consensus clinical diagnosis','sex', 'ADNC'],
                         max_iter_harmony=25, basis='X_pca_harmony_input')


# Access the 'X_pca_harmony' stored in the 'obsm' attribute
harmony_pca = adata.obsm['X_pca_harmony']

# Get the number of components (columns) in the 'X_pca_harmony' matrix
num_components = harmony_pca.shape[1]
print(f'Number of PCA components in X_pca_harmony: {num_components}')


In [ ]:
##### Find neighbors, use leiden clustering, and crate umap
sc.pp.neighbors(adata, use_rep="X_pca_harmony",n_neighbors=15,random_state=11)  #n_neighbors=30 can be used as well
sc.tl.leiden(adata, resolution=0.50, key_added='leiden',random_state=11)  # Adjust resolution as needed

# Changed min_dist for optimal graphing
sc.tl.umap(adata, random_state=11, min_dist=0.15)

In [ ]:
sc.pl.umap(
    adata,
    color="leiden",
    size=2
)

In [ ]:
adata

<div class="alert alert-block alert-info">
<b> Save adata file with UMAP rendering

</div>

In [ ]:
adata.write_h5ad("/tscc/lustre/ddn/scratch/aopatel/adata_mtg_UMAP.h5ad")

#### A2. Find gene markers

In [ ]:
adata=ad.read_h5ad("/tscc/lustre/ddn/scratch/aopatel/adata_mtg_UMAP.h5ad")

In [ ]:
# Assuming adata has Leiden clustering and 'gene_symbols' in adata.var
sc.tl.rank_genes_groups(adata, groupby='leiden', method='t-test')

# Extract and print top 25 markers with gene symbols
result = adata.uns['rank_genes_groups']
for group in result['names'].dtype.names:
    ensembl_ids = result['names'][group][:25]
    symbols = [adata.var['gene_symbols'][adata.var.index.get_loc(ens_id)] for ens_id in ensembl_ids]
    print(f"Cluster {group}:")
    for ens_id, symbol in zip(ensembl_ids, symbols):
        print(f"{ens_id} -> {symbol}")


In [ ]:
sc.pl.umap(adata, color=['RBFOX3'], gene_symbols='gene_symbols',size=2)

In [ ]:
sc.pl.umap(adata, color=['NEUROD1'], gene_symbols='gene_symbols',size=2)